In [ ]:
from fastapi import FastAPI, Query
from typing import Optional, List, Dict
from fuzzywuzzy import fuzz
import csv

app = FastAPI()

# Load company database from CSV
def load_company_db(csv_file: str) -> List[Dict[str, str]]:
    company_db = []
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            company_db.append({
                "name": row.get("name", ""),
                "website": row.get("website", ""),
                "phone": row.get("phone", ""),
                "facebook": row.get("facebook", "")
            })
    return company_db

COMPANY_DB = load_company_db("companies.csv")

def match_company(name: Optional[str], website: Optional[str], phone: Optional[str], facebook: Optional[str]):
    best_match = None
    best_score = 0
    
    for company in COMPANY_DB:
        score = 0
        
        if name:
            score += fuzz.partial_ratio(name.lower(), company["name"].lower())
        if website:
            score += fuzz.partial_ratio(website.lower(), company["website"].lower())
        if phone:
            score += 50 if phone == company["phone"] else 0  # Exact match for phone
        if facebook:
            score += fuzz.partial_ratio(facebook.lower(), company["facebook"].lower())
        
        if score > best_score:
            best_score = score
            best_match = company
    
    return best_match

@app.get("/match_company")
def get_best_match(
    name: Optional[str] = Query(None),
    website: Optional[str] = Query(None),
    phone: Optional[str] = Query(None),
    facebook_profile: Optional[str] = Query(None),
):
    best_match = match_company(name, website, phone, facebook_profile)
    if best_match:
        return {"best_match": best_match}
    return {"message": "No suitable match found"}
